In [ ]:
import os
from os import cpu_count
from math import floor
import pandas as pd
import numpy as np
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [ ]:
data=pd.read_parquet("/kaggle/input/nfunswnb15v2/NF-UNSW-NB15-V2.parquet")

## very very Basic EDA

In [ ]:
data.dtypes

In [ ]:
data.Label.value_counts()

In [ ]:
data.Attack.value_counts()

In [ ]:
data=data.drop(columns=['L4_SRC_PORT', 'L4_DST_PORT']) #dropping metadata

## Finding contaminant features using XAI (SHAP)
goal: 
- try to find the contaminant features that have blanked predictive power across all attack classes

method:
* look for shap values per feature that are consistantly high or low among all attack classes.
   
how:
```
possible_contaminants =[]
    while models can achive high accuracy{
    
        * Train a binary classification model (e.g., xgboost) on different attack classes and benign class
        * For each model, calculate shappley values on test set
        * For each model, use np.abs(shap_values).mean(0) to compute the mean absolute SHAP value for each feature across all samples. 
          This will give you a measure of how much each feature contributes to the model output on average across all samples (the importance).
          
        * normalize importances across for each attack class sum of feature importances equals 1
        * for each feature, calculate variance of importance for each attack class
        * for each feature, calculate average importance across all attack classes
        * score of a feature is the weighted sum of the importance and variance
        
        * feature F is feature with the highest score
        * possible_contaminants.append(F)
        * drop F from dataset
    }
```

In [ ]:
training_set = data.sample(frac=0.3, replace=False,random_state=42)
testing_set = data.drop(index=training_set.index)

In [ ]:
training_set.Attack.value_counts()

In [ ]:
testing_set=testing_set[testing_set.Attack!="Worms"]

In [ ]:
training_set=training_set[training_set.Attack!="Worms"]
attacks=training_set.Attack.unique()
attacks

### split into dataframes per attack class

In [ ]:
grouped = training_set.groupby(training_set.Attack)
dfs={cat:grouped.get_group(cat) for cat in attacks[1:]}
dfs[attacks[0]]=grouped.get_group(attacks[0]) #don't subsample normal attacks
#dfs key=attack_cat, value is dataframe 

In [ ]:
for atk_type in attacks[1:]:
    normals_to_sample = dfs[atk_type].shape[0]
#     print(normals_to_sample)
    normals_sample = dfs['Benign'].sample(normals_to_sample)
#     dfs['Normal'] = dfs['Normal'].drop(index=normals_sample.index)#don't resample
    dfs[atk_type] = pd.concat(objs=[dfs[atk_type], normals_sample])
    print(dfs[atk_type].shape[0], normals_sample.shape[0], dfs['Benign'].shape[0])

In [ ]:
testing_dfs = {}
grouped = testing_set.groupby(testing_set.Attack)
testing_dfs={cat:grouped.get_group(cat) for cat in attacks[1:]}
testing_dfs[attacks[0]]=grouped.get_group(attacks[0]) #don't subsample normal attacks
#testing_dfs key=attack_cat, value is dataframe 

In [ ]:
for atk_type in attacks[1:]:
    normals_to_sample = testing_dfs[atk_type].shape[0] 
    normals_sample = testing_dfs['Benign'].sample(normals_to_sample)
#     testing_dfs['Normal'] = testing_dfs['Normal'].drop(index=normals_sample.index)#don't resample
    testing_dfs[atk_type] = pd.concat(objs=[testing_dfs[atk_type], normals_sample])
    print(testing_dfs[atk_type].shape[0], normals_sample.shape[0], testing_dfs['Benign'].shape[0])

In [ ]:
print(f"TRAINING SETS")
for k,v in dfs.items():
    v.drop(columns=['Attack'], inplace=True)   
    print(k, v.shape)
    print(v['Label'].value_counts())

In [ ]:
print("TESTING SETS")
for k,v in testing_dfs.items():
    v.drop(columns=['Attack'], inplace=True)   
    print(k, v.shape)
    print(v['Label'].value_counts())

In [ ]:
training_dfs=dfs

In [ ]:
import plotly.express as px

### Min & Max TTL

In [ ]:
np.corrcoef(training_set.MIN_TTL,training_set.MAX_TTL)

In [ ]:
px.histogram(training_set,x='MIN_TTL',color='Label',barmode='group')

In [ ]:
training_set.groupby(["Label","MIN_TTL"]).MIN_TTL.count()

In [ ]:
training_set.groupby('Attack')['MIN_TTL'].apply(lambda x: ((x ==0)|(x ==62)|(x ==254)).mean() * 100)

### MIN_IP_PKT_LEN

In [ ]:
px.histogram(training_set,x='MIN_IP_PKT_LEN',color='Label',barmode='group',histnorm="percent")

In [ ]:
training_set.groupby('Attack')['MIN_IP_PKT_LEN'].apply(lambda x: ((x ==0)|(x ==40)|(x ==48)).mean() * 100)

In [ ]:
training_set.groupby(["Label","MIN_IP_PKT_LEN"]).Label.count()

In [ ]:
px.histogram(
    training_set,
    x='MIN_IP_PKT_LEN',
    color='Label',
    marginal='box',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent' 
)

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],
    x='MIN_IP_PKT_LEN',
    color='Label',
    marginal='box',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

### SHORTEST_FLOW_PKT

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],
    x='SHORTEST_FLOW_PKT',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

### TCP_FLAGS & CLIENT_TCP_FLAGS , SERVER_TCP_FLAGS

In [ ]:
np.corrcoef([training_set.TCP_FLAGS,training_set.CLIENT_TCP_FLAGS,training_set.SERVER_TCP_FLAGS])

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],
    x='TCP_FLAGS',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.groupby('Attack')['TCP_FLAGS'].apply(lambda x: ((x ==0)|(x ==27)|(x ==19)).mean() * 100)

### TCP_WIN_MAX_OUT

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],
    x='TCP_WIN_MAX_OUT',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.groupby('Attack')['TCP_WIN_MAX_OUT'].apply(lambda x: ((x ==0)|(x ==16383)).mean() * 100)

### TCP_WIN_MAX_IN

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],
    x='TCP_WIN_MAX_IN',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.groupby('Attack')['TCP_WIN_MAX_IN'].apply(lambda x: ((x ==0)|(x ==16383)).mean() * 100)

### DST_TO_SRC_SECOND_BYTES

In [ ]:
import math
for attack in attacks[1:]:
    d=training_dfs[attack][training_dfs[attack].DST_TO_SRC_SECOND_BYTES<=1000000]
    k=d[d.Label==1]
    bin_width= 100
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["DST_TO_SRC_SECOND_BYTES"].max() - k["DST_TO_SRC_SECOND_BYTES"].min()) / bin_width)
    fig=px.histogram(
    d,
    x='DST_TO_SRC_SECOND_BYTES',
    color='Label',nbins=nbins,
    marginal='box',
#     barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.groupby('Attack')['DST_TO_SRC_SECOND_BYTES'].apply(lambda x: (x <1000).mean() * 100)

### OUT_BYTES

In [ ]:
import math
for attack in attacks[1:]:
    d=training_dfs[attack][training_dfs[attack].DST_TO_SRC_SECOND_BYTES<=1000000]
    k=d[d.Label==1]
    bin_width= 100
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["OUT_BYTES"].max() - k["OUT_BYTES"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='OUT_BYTES',
    color='Label',
    marginal='box',
    barmode='group',nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.groupby('Attack')['OUT_BYTES'].apply(lambda x: (x <1000).mean() * 100)

In [ ]:
px.histogram(training_set[~training_set.Attack.isin(['Generic','DoS','Exploits'])],x='OUT_PKTS',color='Label',barmode='group',marginal='box')

In [ ]:
px.histogram(training_set[training_set.Attack.isin(['Benign'])],x='OUT_BYTES',color='Label',barmode='group',marginal='box')

In [ ]:
training_set.groupby('Attack')['OUT_BYTES'].apply(lambda x: (x ==0).mean() * 100)

In [ ]:
training_set.groupby('Attack')['OUT_PKTS'].apply(lambda x: (x ==0).mean() * 100)

In [ ]:
training_set.groupby('Attack')['IN_PKTS'].apply(lambda x: (x ==0).mean() * 100)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(4, 2, figsize=(12, 12))
axes = axes.flatten()

colors = ["#00FF00","#FF0000"]

for i, attack in enumerate(attacks[1:]):
    sns.histplot(training_dfs[attack],
                 x='LONGEST_FLOW_PKT',
                 hue='Label',
                 stat='percent',palette=colors,
                 ax=axes[i])
    axes[i].set_title(attack)
plt.tight_layout()
plt.show()

### DST_TO_SRC_AVG_THROUGHPUT

In [ ]:
import math
for attack in attacks[1:]:
#     d=training_dfs[attack][training_dfs[attack].DST_TO_SRC_SECOND_BYTES<=1000000]
    k=training_dfs[attack][training_dfs[attack].Label==1]
    k=k[k.DST_TO_SRC_AVG_THROUGHPUT>0]
    bin_width= 100000
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["DST_TO_SRC_AVG_THROUGHPUT"].max() - k["DST_TO_SRC_AVG_THROUGHPUT"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='DST_TO_SRC_AVG_THROUGHPUT',
    color='Label',
    marginal='box',
    barmode='group',nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
k=training_set[["DST_TO_SRC_AVG_THROUGHPUT","DST_TO_SRC_SECOND_BYTES"]]
k["new"]=k["DST_TO_SRC_AVG_THROUGHPUT"]/8000
k

In [ ]:
(k["DST_TO_SRC_SECOND_BYTES"]==k["new"]).value_counts()

In [ ]:
np.corrcoef(k.new,k.DST_TO_SRC_SECOND_BYTES)

In [ ]:
training_set.groupby('Attack')['DST_TO_SRC_SECOND_BYTES'].apply(lambda x: (x ==0).mean() * 100)

### NUM_PKTS_UP_TO_128_BYTES

In [ ]:
training_set.groupby('Attack')['NUM_PKTS_UP_TO_128_BYTES'].apply(lambda x: (x<20).mean() * 100)

In [ ]:
import math 
for attack in attacks[1:]:
#     d=training_dfs[attack][training_dfs[attack].DST_TO_SRC_SECOND_BYTES<=1000000]
    k=training_dfs[attack][training_dfs[attack].Label==1]
    k=k[k.NUM_PKTS_UP_TO_128_BYTES>0]
    bin_width= 100
#     here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["NUM_PKTS_UP_TO_128_BYTES"].max() - k["NUM_PKTS_UP_TO_128_BYTES"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='NUM_PKTS_UP_TO_128_BYTES',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

### IN_PKTS

In [ ]:
import math
for attack in attacks[1:]:
#     d=training_dfs[attack][training_dfs[attack].DST_TO_SRC_SECOND_BYTES<=1000000]
    k=training_dfs[attack][training_dfs[attack].Label==1]
#     k=k[k.DST_TO_SRC_AVG_THROUGHPUT>0]
    bin_width= 5
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["IN_PKTS"].max() - k["IN_PKTS"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='IN_PKTS',
    color='Label',
    marginal='box',
    barmode='group',nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
(training_set.groupby('Attack')['IN_PKTS'].apply(lambda x: ((x.between(0,4)) | (x.between(10,14))).mean())*100).reset_index(name='Percentage')

### LONGEST_FLOW_PKT

In [ ]:
import math
for attack in attacks[1:]:
#     d=training_dfs[attack][training_dfs[attack].DST_TO_SRC_SECOND_BYTES<=1000000]
    k=training_dfs[attack]#[training_dfs[attack].Label==1]
#     k=k[k.DST_TO_SRC_AVG_THROUGHPUT>0]
    bin_width= 10
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["LONGEST_FLOW_PKT"].max() - k["LONGEST_FLOW_PKT"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='LONGEST_FLOW_PKT',
    color='Label',
    marginal='box',
    barmode='group',#nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

### SRC_TO_DST_SECOND_BYTES

In [ ]:
import math
for attack in attacks[1:]:
    d=training_dfs[attack][training_dfs[attack].SRC_TO_DST_SECOND_BYTES<=1000000]
    k=d#[training_dfs[attack].Label==1]
#     k=k[k.DST_TO_SRC_AVG_THROUGHPUT>0]
    bin_width= 100
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["SRC_TO_DST_SECOND_BYTES"].max() - k["SRC_TO_DST_SECOND_BYTES"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='SRC_TO_DST_SECOND_BYTES',
    color='Label',
    marginal='box',
    barmode='group',nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
(training_set.groupby('Attack')['SRC_TO_DST_SECOND_BYTES'].apply(lambda x: ((x<800)|(x>8900)).mean())*100).reset_index(name='Percentage')

### IN_BYTES 'RETRANSMITTED_IN_BYTES', 'RETRANSMITTED_IN_PKTS'

In [ ]:
import math
for attack in attacks[1:]:
    d=training_dfs[attack]#[training_dfs[attack].SRC_TO_DST_SECOND_BYTES<=1000000]
    k=d#[training_dfs[attack].Label==1]
#     k=k[k.DST_TO_SRC_AVG_THROUGHPUT>0]
    bin_width= 500
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["IN_BYTES"].max() - k["IN_BYTES"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='IN_BYTES',
    color='Label',
    marginal='box',
    barmode='group',nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.groupby('Attack')['IN_BYTES'].apply(lambda x: (x<1000).mean() * 100)

### SRC_TO_DST_AVG_THROUGHPUT

In [ ]:
import math
for attack in attacks[1:]:
    d=training_dfs[attack][training_dfs[attack].SRC_TO_DST_AVG_THROUGHPUT<=1000000]
    k=d#[d.Label==1]
#     k=k[k.DST_TO_SRC_AVG_THROUGHPUT>0]
    bin_width= 100000
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["SRC_TO_DST_AVG_THROUGHPUT"].max() - k["SRC_TO_DST_AVG_THROUGHPUT"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='SRC_TO_DST_AVG_THROUGHPUT',
    color='Label',
    marginal='box',
    barmode='group',#nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
(training_set.groupby('Attack')['SRC_TO_DST_AVG_THROUGHPUT'].apply(lambda x: ((x>7000000)).mean())*100).reset_index(name='Percentage')

### ICMP_TYPE

In [ ]:
import math
for attack in attacks[1:]:
    d=training_dfs[attack]#[training_dfs[attack].ICMP_TYPE<=1000000]
    k=d#[training_dfs[attack].Label==1]
    k=k#[k.ICMP_TYPE>0]
    bin_width= 1000
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["ICMP_TYPE"].max() - k["ICMP_TYPE"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='ICMP_TYPE',
    color='Label',
    marginal='box',
    barmode='group',#nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.groupby('Attack')['ICMP_TYPE'].apply(lambda x: (x==0).mean() * 100)

### L7_PROTO

In [ ]:
import math
for attack in attacks[1:]:
    d=training_dfs[attack]#[training_dfs[attack].ICMP_TYPE<=1000000]
    k=d[training_dfs[attack].Label==1]
#     k=k[k.DST_TO_SRC_AVG_THROUGHPUT>0]
    bin_width= 5
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["L7_PROTO"].max() - k["L7_PROTO"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='L7_PROTO',
    color='Label',
    marginal='box',
    barmode='group',nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.groupby('Attack')['L7_PROTO'].apply(lambda x: (x==0).mean() * 100)

### PROTOCOL

In [ ]:
import math
for attack in attacks[1:]:
    d=training_dfs[attack]#[training_dfs[attack].ICMP_TYPE<=1000000]
    k=d#[training_dfs[attack].Label==1]
#     k=k[k.DST_TO_SRC_AVG_THROUGHPUT>0]
    bin_width= 1000
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["PROTOCOL"].max() - k["PROTOCOL"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='PROTOCOL',
    color='Label',
    marginal='box',
    barmode='group',#nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_dfs["Benign"].PROTOCOL.value_counts()

In [ ]:
training_set.groupby('Attack')['PROTOCOL'].apply(lambda x: ((x==6)|(x==17)).mean() * 100)

# Random

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],
    x='FLOW_DURATION_MILLISECONDS',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
k=training_set.FLOW_DURATION_MILLISECONDS.value_counts()

In [ ]:
k[k>10]

In [ ]:
(training_set.groupby('Attack')['FLOW_DURATION_MILLISECONDS'].apply(lambda x: ((x>0)).mean())*100).reset_index(name='Percentage')

### CLIENT_TCP_FLAGS

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],
    x='CLIENT_TCP_FLAGS',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.groupby('Attack')['CLIENT_TCP_FLAGS'].apply(lambda x: ((x ==19)|(x ==0)|(x ==27)).mean() * 100)

### DURATION_OUT

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],
    x='DURATION_OUT',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.groupby('Attack')['DURATION_OUT'].apply(lambda x: ((x ==0)|(x ==0)|(x ==0)).mean() * 100)

### NUM_PKTS_128_TO_256_BYTES

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],
    x='NUM_PKTS_128_TO_256_BYTES',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.groupby('Attack')['NUM_PKTS_128_TO_256_BYTES'].apply(lambda x: ((x ==0)|(x ==2)|(x ==0)).mean() * 100)

## SERVER_TCP_FLAGS

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],
    x='SERVER_TCP_FLAGS',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.groupby('Attack')['SERVER_TCP_FLAGS'].apply(lambda x: ((x ==0)|(x ==19)|(x ==0)).mean() * 100)

### NUM_PKTS_512_TO_1024_BYTES

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],
    x='NUM_PKTS_512_TO_1024_BYTES',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.groupby('Attack')['NUM_PKTS_512_TO_1024_BYTES'].apply(lambda x: ((x ==0)|(x ==0)|(x ==0)).mean() * 100)

### NUM_PKTS_256_TO_512_BYTES

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],
    x='NUM_PKTS_256_TO_512_BYTES',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.groupby('Attack')['NUM_PKTS_256_TO_512_BYTES'].apply(lambda x: ((x ==0)|(x ==2)|(x ==0)).mean() * 100)

### KS

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(4, 2, figsize=(12, 12))
axes = axes.flatten()

colors = ["#00FF00","#FF0000"]

for i, attack in enumerate(attacks[1:]):
    sns.histplot(training_dfs[attack],
                 x='LONGEST_FLOW_PKT',
                 hue='Label',
                 stat='percent',palette=colors,
                 ax=axes[i])
    axes[i].set_title(attack)
plt.tight_layout()
plt.show()